In [ ]:
import openai
import pandas as pd
import time
import requests
import json
import jsonlines
import re

sk = "YOUR API KEY"
openai.api_key = sk

def chat(input_data, model="gpt-3.5-turbo", temperature=0.8):
    
    nmessages = [{"role": "user", "content": input_data, "temperature": temperature}]

    while (1):
        try:
            response = openai.ChatCompletion.create(          
                model=model,
                messages=nmessages
            )
            resmessage = response['choices'][0]['message']['content']
            break
        except:
            time.sleep(10)
            
    return  resmessage

In [ ]:
def convert_to_table(serialized_tuple):
    # 分割标题和数据
    caption_split = serialized_tuple.split(' attribute ')
    title = caption_split[0].split(']: ')[1].strip()

    # 提取属性和值
    attributes = caption_split[1:]

    headers = []
    values = []
    sign = 0
    

    for attribute in attributes:
        attribute_value_split = attribute.split(' value ')
        attribute_name = attribute_value_split[0].strip()
        value = attribute_value_split[1].split(' attribute ')[0].strip()  # 分割可能的下一个属性
        
        if sign == 1 and len(attribute_name) > 10:
            attribute_name = attribute_name[:10]
        headers.append(attribute_name)
        
        values.append(value)

    # 构建表格
    table = 'caption: ' + title + '\n|' + ' | '.join(headers) + ' |\n|' + ' | '.join(values) + ' |'
    return table


In [ ]:
processed_tuples = []
with open('./results/education/GPT4_education_wo_evidence.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        tuple_id = int(line['tuple_id'])
        processed_tuples.append(tuple_id)

print(len(processed_tuples))

In [ ]:
import json
import jsonlines
import pandas as pd
import random
import ast

template = '''What's the most likely value for the [TO-FILL] cell in the table below? Please respond using JSON: {'Street Address': '', 'ZIP Code': '', 'Phone Number': ''}, the key is attribute name of each [TO-FILL], value is the predicted value for each [TO-FILL].\n'''


tableData = []

with open('/Users/yichendezaizai/Data_Imputation/data/education/annotated_data/folds.json', 'r') as f:
    folds = json.load(f)
    test_qids = folds['test']
print(len(test_qids))


count, acc = 0, 0


queries = {}
with open('/Users/yichendezaizai/Data_Imputation/data/education/annotated_data/queries.tsv', 'r') as f:
    for line in f:
        line = line.strip()
        qid, query = int(line[:line.index('\t')]), line[line.index('\t')+1:]
        query_text = convert_to_table(query)
        query_text = query_text.replace('N/A', '[TO-FILL]')
        queries[qid] = query_text

columns = ['Name of School', 'Street Address', 'City', 'State', 'ZIP Code', 'Phone Number', 'Website', 'Teacher Response Rate', 'Student Response Rate']
missing_columns = ['Street Address', 'ZIP Code', 'Phone Number']

example = ["Walter Q Gresham Elementary School", "8524 S Green St", "Chicago", "IL", "60620", "(773) 535-3350", "http://www.cps.edu" "100.0", "84.9"]
for qid, query in queries.items():

    if qid in processed_tuples or qid not in test_qids:
        continue
    
    input_data = template

    input_data += query + '\n|' + ' | '.join(example) + ' |'
    print("-------------------")
    print(input_data)

    output = chat(input_data, model="gpt-4", temperature=0.3) #    gpt-3.5-turbo; gpt-35-turbo-1106
    print(output)

    fout = jsonlines.open('./results/education/GPT4_education_wo_evidence.jsonl', 'a')
    fout.write({'tuple_id':qid, 'input': input_data, 'output': output})
    fout.close()


## Imputed Data with retrieved tables

In [ ]:
top_K = 5

In [ ]:
from collections import defaultdict

all_scores = defaultdict(dict)
#with open('/Users/yichendezaizai/Data_Imputation/retrieval_results/rerank_results/education_neg11.test.tsv', 'r') as f:
with open('/Users/yichendezaizai/Data_Imputation/retrieval_results/first_stage/BM25_top100_res_with_score_education.tsv', 'r') as f:
    for line in f:
        qid, docid, rank, score = line.strip().split('\t')
        #qid, docid, score = line.strip().split('\t')
        score = float(score)
        all_scores[int(qid)][int(docid)] = score

qq = list(all_scores.keys())

# topK_pids
topK_results = {}
for qid in qq:
    score_list = sorted(list(all_scores[qid].items()), key=lambda x: x[1], reverse=True)
    for rank, (docid, score) in enumerate(score_list):
        if rank >= top_K:
            continue
        if qid not in topK_results:
            topK_results[qid] = []
        topK_results[qid].append(docid)


In [ ]:
collection = {}
with open('/Users/yichendezaizai/Data_Imputation/data/education/annotated_data/collection.tsv', 'r') as f:
    for line in f:
        line = line.strip()
        qid, query = line[:line.find('\t')], line[line.find('\t')+1:]
        collection[int(qid)] = query

In [ ]:
processed_tuples = []
with open('/Users/yichendezaizai/Data_Imputation/imputation/results/education/GPT4_education_with_retrieved_tuples_by_BM25.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        tuple_id = int(line['tuple_id'])
        processed_tuples.append(tuple_id)

print(len(processed_tuples))

In [ ]:
with open('/Users/yichendezaizai/Data_Imputation/imputation/results/education/ablation/test_ids.json', 'r') as f:
    test_qids = json.load(f)

In [ ]:
with open('/Users/yichendezaizai/Data_Imputation/data/education/annotated_data/folds.json', 'r') as f:
    folds = json.load(f)
    test_qids = folds['test']
print(len(test_qids))

In [ ]:
import json
import jsonlines
import pandas as pd
import random
import ast

template = '''Based on the retrieved tabular data, what's the most likely value for the [TO-FILL] cell in the table below? Please respond using JSON: {'Street Address': '', 'ZIP Code': '', 'Phone Number': ''}, the key is attribute name of each [TO-FILL], value is the predicted value for each [TO-FILL].\n'''


tableData = []
count, acc = 0, 0

queries = {}
with open('/Users/yichendezaizai/Data_Imputation/data/education/annotated_data/queries.tsv', 'r') as f:
    for line in f:
        line = line.strip()
        qid, query = int(line[:line.index('\t')]), line[line.index('\t')+1:]
        query_text = convert_to_table(query)
        query_text = query_text.replace('N/A', '[TO-FILL]')
        queries[qid] = query_text

columns = ['Name of School', 'Street Address', 'City', 'State', 'ZIP Code', 'Phone Number', 'Website', 'Teacher Response Rate', 'Student Response Rate']
missing_columns = ['Street Address', 'ZIP Code', 'Phone Number']

#[Caption]: Public Elementary School  attribute Name of School value Walter Q Gresham Elementary School attribute Street Address value N/A attribute City value Chicago attribute State value IL attribute ZIP Code value N/A attribute Phone Number value N/A attribute Website value http://www.cps.edu attribute Teacher Response Rate value 100.0 attribute Student Response Rate value 84.9

for qid, query in queries.items():
    
    if qid not in test_qids or qid in processed_tuples:
        continue

    input_data = template + query + '\n|'

    # Adding retrieved tables
    input_data += 'Retrieved Tables:\n'
    retrieved_tables = topK_results[qid]
    for rank, docid in enumerate(retrieved_tables):
        input_data += 'Table ' + str(rank+1) + ': ' + convert_to_table(collection[docid]) + '\n\n'


    print("-------------------")
    print(input_data)

    output = chat(input_data, model="gpt-4", temperature=0.3)
    print(output)

    fout = jsonlines.open('/Users/yichendezaizai/Data_Imputation/imputation/results/education/GPT4_education_with_retrieved_tuples_by_BM25.jsonl', 'a')
    fout.write({'tuple_id':qid, 'input': input_data, 'output': output})
    fout.close()



### Calculate scores

In [ ]:
import json
import jsonlines
import pandas as pd
import random
import ast

template = '''Based on the retrieved tabular data, what's the most likely value for the [TO-FILL] cell in the table below? Please respond using JSON: {'Street Address': '', 'ZIP Code': '', 'Phone Number': ''}, the key is attribute name of each [TO-FILL], value is the predicted value for each [TO-FILL].\n'''


tableData = []
ground_truth = {}
with jsonlines.open('/Users/yichendezaizai/Data_Imputation/data/education/annotated_data/answer.jsonl', 'r') as f:
    for line in f:
        ground_truth[line['query_id']] = line['answers']

imputed_record = {}
with open('/Users/yichendezaizai/Data_Imputation/imputation/results/education/GPT4_education_with_retrieved_tuples_by_BM25.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        tuple_id = int(line['tuple_id'])
        imputed_record[tuple_id] = ast.literal_eval(line['output'])

        # imputed_record[line['tuple_id']] = ast.literal_eval(line['output'])
            


count, acc = 0, 0

queries = {}
with open('/Users/yichendezaizai/Data_Imputation/data/education/annotated_data/queries.tsv', 'r') as f:
    for line in f:
        line = line.strip()
        qid, query = int(line[:line.index('\t')]), line[line.index('\t')+1:]
        query_text = convert_to_table(query)
        query_text = query_text.replace('N/A', '[TO-FILL]')
        queries[qid] = query_text

columns = ['Name of School', 'Street Address', 'City', 'State', 'ZIP Code', 'Phone Number', 'Website', 'Teacher Response Rate', 'Student Response Rate']
missing_columns = ['Street Address', 'ZIP Code', 'Phone Number']

total_queries = 0
for qid, query in queries.items():
    
    if qid not in test_qids:
        continue

    total_queries  += 1

    # 处理输出的数据
    try:
        imputed_data = imputed_record[qid]
    except:
        imputed_data = {}

    correct_values = {}
    for key, value in ground_truth[qid].items():
        correct_values[key] = []
        for vv in value:
            correct_values[key].append(vv.lower().replace('(','').replace(')',''))

    for i, col in enumerate(missing_columns):
        count += 1
        if col not in imputed_data:
            continue
        if imputed_data[col].lower().replace('(','').replace(')','') in correct_values[col]:
            acc += 1
        

accuaracy = round(acc/count, 3)
print(f"Imputed Accuracy: {accuaracy}")
print(total_queries)
